In [16]:
import matplotlib.pyplot as plt
%matplotlib inline
import skdsp.signal.discrete as ds
import numpy as np
import sympy as sp
import scipy.fftpack as fftp
from ipywidgets import interactive, IntSlider, FloatSlider, Checkbox, ToggleButton, ToggleButtons, HBox, VBox
from IPython.display import display

In [23]:
def dsfdemo(signal='Diente de sierra', N=8, width=0, offset=0, shift=0, center=True):
    if width > N:
        raise ValueError('width debe ser menor que N')
    if signal == 'Diente de sierra':
        s = ds.Sawtooth(N, width)
    elif signal == 'Cuadrada':
        s = ds.Square(N, width)
    s = offset + (s >> shift)
    npts = 32
    n = np.arange(-npts, npts+1)
    plt.figure(figsize=(20, 7))
    plt.subplot(2, 2, (1, 2))
    plt.stem(n, s[-npts:npts+1], markerfmt='bo', linefmt='b-', basefmt='k-')
    plt.axis([-npts-0.5, npts+0.5, -2.1, 2.1])
    plt.axvline(0, color='k')
    plt.axhline(0, color='k')
    plt.grid(True)
    X = s.dfs()
    X.real[np.isclose(np.real(X), 0)] = 0
    X.imag[np.isclose(np.imag(X), 0)] = 0
    if center:
        X = fftp.fftshift(X)
        kidx = np.arange(-int(N/2), int((N-1)/2)+1)
    else:
        kidx = np.arange(0, N)
    plt.subplot(2, 2, 3)
    mX = np.abs(X)
    plt.stem(kidx, mX)
    min_ = min(0, np.min(mX))
    max_ = np.max(mX)
    dr = min(1.0, (max_ - min_))
    plt.axis([kidx[0]-0.5, kidx[-1]+0.5, min_-0.15*dr, max_+0.15*dr])
    plt.axhline(0, color='k')
    plt.axvline(0, color='k')
    plt.grid(True)
    plt.subplot(2, 2, 4)
    plt.stem(kidx, np.angle(X))
    plt.axis([kidx[0]-0.5, kidx[-1]+0.5, -np.pi-0.5, np.pi+0.5])
    plt.axhline(0, color='k')
    plt.axvline(0, color='k')
    plt.grid(True)

In [24]:
Nw = IntSlider(value=16, min=2, max=32, step=1, continuous_update=False, description='N:')
kw = IntSlider(value=1, min=1, max=32, step=1, continuous_update=False, description='width:')
ow = FloatSlider(value=0, min=-1.0, max=1.0, step=0.1, continuous_update=False, description='offset:')
sw = IntSlider(value=0, min=-64, max=64, step=1, continuous_update=False, description='shift:')
cw = ToggleButton(value=True, description='DFS centrada')
ww = ToggleButtons(options=['Diente de sierra', 'Cuadrada'], description='Señal:')
w = interactive(dsfdemo, signal=ww, N=Nw, width=kw, offset=ow, shift=sw, center=cw)

In [25]:
display(HBox(children=[VBox(children=[ww, Nw, kw]), VBox(children=[cw, ow, sw])]))